# SH expansion analytically

In [8]:
from sympy import symbols, integrate, sin, cos, exp, pi, sqrt, Ynm, re, im

# Define the variables
theta, phi = symbols('theta phi')

# Define the function f(theta, phi) in terms of exponential
#f_exp = sin(theta) * (exp(1j*phi) + exp(-1j*phi)) / 2
f_exp = sin(theta) * cos(phi)

# Function to calculate the spherical harmonic coefficient
def calculate_coefficient(l, m, f):
    Y_lm = Ynm(l, m, theta, phi).expand(func=True)
    integrand = f * Y_lm.conjugate() * sin(theta)
    a_lm_real_part = integrate(re(integrand), (theta, 0, pi), (phi, 0, 2*pi))
    a_lm_imag_part = integrate(im(integrand), (theta, 0, pi), (phi, 0, 2*pi))
    return a_lm_real_part, a_lm_imag_part

# Calculating coefficients for l=0 to l=2
coefficients = {}
for l in range(3):
    for m in range(-l, l + 1):
        coefficients[(l, m)] = calculate_coefficient(l, m, f_exp)

coefficients


{(0, 0): (0, 0),
 (1, -1): (sqrt(6)*sqrt(pi)/3, 0),
 (1, 0): (0, 0),
 (1, 1): (-sqrt(6)*sqrt(pi)/3, 0),
 (2, -2): (0, 0),
 (2, -1): (0, 0),
 (2, 0): (0, 0),
 (2, 1): (0, 0),
 (2, 2): (0, 0)}

# Numerical epxpansion

In [9]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

# Function to be expanded: sin(theta) * cos(phi)
def function(theta, phi):
    return np.sin(theta) * np.cos(phi)

# Spherical harmonics expansion coefficients
def spherical_harmonic_coeff(l, m):
    def integrand(phi, theta):
        # Spherical harmonic Y_lm(theta, phi)
        Y_lm = special.sph_harm(m, l, phi, theta)
        return function(theta, phi) * np.conj(Y_lm) * np.sin(theta)

    # Integration over theta [0, pi] and phi [0, 2*pi]
    return integrate.nquad(integrand, [[0, np.pi], [0, 2*np.pi]])[0]

# Range of l and m for the expansion
l_max = 5

# Calculate coefficients
coefficients = {}
for l in range(l_max + 1):
    for m in range(-l, l + 1):
        coeff = spherical_harmonic_coeff(l, m)
        if np.abs(coeff) > 1e-10:  # Ignoring very small coefficients for clarity
            coefficients[(l, m)] = coeff

coefficients



C:\Users\MartinKaras(AI)\.conda\envs\n_body_approx\lib\site-packages\scipy\integrate\_quadpack_py.py:575: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


{(1, -1): 1.4472025091165355, (1, 1): -1.4472025091165355}

# numerical expansion, manual sampling

In [10]:
# Number of sample points in theta and phi directions
n_theta = 40
n_phi = 80

# Generate sample points
theta_samples = np.linspace(0, np.pi, n_theta)
phi_samples = np.linspace(0, 2 * np.pi, n_phi)

# Evaluate the function at each sample point
function_samples = np.array([[function(theta, phi) for phi in phi_samples] for theta in theta_samples])

# Initialize a dictionary to store coefficients
coefficients_manual = {}

# Calculate coefficients manually
for l in range(l_max + 1):
    for m in range(-l, l + 1):
        # Evaluate spherical harmonics at each sample point
        Y_lm_samples = np.array([[special.sph_harm(m, l, phi, theta) for phi in phi_samples] for theta in theta_samples])

        # Approximate the coefficient a_lm
        product = function_samples * np.conj(Y_lm_samples) * np.sin(theta_samples)[:, np.newaxis]
        coeff = np.sum(product) * (np.pi / n_theta) * (2 * np.pi / n_phi)  # Multiplying by the area element

        if np.abs(coeff) > 1e-10:  # Ignoring very small coefficients for clarity
            coefficients_manual[(l, m)] = coeff

coefficients_manual



{(0, 0): (0.03393199873069333+0j),
 (1, -1): (1.428660980066449-2.072447717324013e-17j),
 (1, 1): (-1.428660980066449-2.072447717324013e-17j),
 (2, -2): (0.03484747711132016+6.163572403142541e-18j),
 (2, 0): (-0.009484281971783352+0j),
 (2, 2): (0.03484747711132018+3.483060446567529e-18j),
 (3, -3): (0.03407942971023795-4.32123038568357e-17j),
 (3, -1): (2.8287634503052517e-06+5.366374236416639e-18j),
 (3, 1): (-2.8287634503490814e-06-2.3434415907781536e-18j),
 (3, 3): (-0.03407942971023795-4.317459034771613e-17j),
 (4, -4): (0.03326900034184693-4.452476323377711e-17j),
 (4, -2): (0.005029800072700013-6.420387919940147e-20j),
 (4, 0): (-0.0015905624405012431+0j),
 (4, 2): (0.005029800072700008-5.221915508217986e-18j),
 (4, 4): (0.03326900034184694+3.716271841891241e-17j),
 (5, -5): (0.03249511914587252-1.7477069460069017e-16j),
 (5, -3): (0.008073476794919-7.994553343540055e-18j),
 (5, -1): (5.646089253799144e-06+6.891216367402424e-18j),
 (5, 1): (-5.646089253875847e-06+5.8639543002120

# Ugly function numerical SH expansion

In [11]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

# Function to be expanded in terms of theta and phi
def function_spherical(theta, phi):
    return (np.sin(theta)**3 * np.cos(phi)**3) / np.exp(np.sin(theta) * np.sin(phi))

# Spherical harmonics expansion coefficients
def spherical_harmonic_coeff(l, m, func):
    def integrand(phi, theta):
        # Spherical harmonic Y_lm(theta, phi)
        Y_lm = special.sph_harm(m, l, phi, theta)
        return func(theta, phi) * np.conj(Y_lm) * np.sin(theta)

    # Integration over theta [0, pi] and phi [0, 2*pi]
    return integrate.nquad(integrand, [[0, np.pi], [0, 2*np.pi]])[0]

# Range of l and m for the expansion
l_max = 10

# Calculate coefficients
coefficients_spherical = {}
for l in range(l_max + 1):
    for m in range(-l, l + 1):
        coeff = spherical_harmonic_coeff(l, m, function_spherical)

        coefficients_spherical[(l, m)] = coeff

coefficients_spherical



{(0, 0): 2.7297309044538403e-17,
 (1, -1): 0.9320936868986063,
 (1, 0): -1.8909069314855235e-18,
 (1, 1): -0.9320936868986063,
 (2, -2): 8.468396730909905e-18,
 (2, -1): 4.788951286468991e-18,
 (2, 0): -1.7303817402801495e-17,
 (2, 1): -2.941476049926174e-17,
 (2, 2): 1.1594652399083913e-17,
 (3, -3): 0.26437990607135164,
 (3, -2): 1.2154364556855747e-18,
 (3, -1): -0.2587483254005577,
 (3, 0): 4.011666887455601e-18,
 (3, 1): 0.2587483254005577,
 (3, 2): 1.5179316159542096e-18,
 (3, 3): -0.26437990607135164,
 (4, -4): -7.721452261421655e-17,
 (4, -3): 2.3505657094580146e-17,
 (4, -2): -1.0798898955069175e-18,
 (4, -1): -5.964363887538337e-17,
 (4, 0): 1.1209738830144967e-17,
 (4, 1): 8.951613538074697e-17,
 (4, 2): -1.0686502483684074e-18,
 (4, 3): -2.3957643835915662e-17,
 (4, 4): -7.518525863134607e-17,
 (5, -5): -0.03431227331854357,
 (5, -4): 3.1606485252313934e-19,
 (5, -3): 0.005514067310438473,
 (5, -2): 3.837825835553818e-18,
 (5, -1): 0.004920288981738131,
 (5, 0): 2.775772299

# Plot shape difference

In [12]:
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Function to compute the spherical harmonics approximation
def spherical_harmonics_approx(theta, phi, coefficients, l_max):
    approx = np.zeros(theta.shape, dtype=complex)
    for l in range(l_max + 1):
        for m in range(-l, l + 1):
            if (l, m) in coefficients:
                approx += coefficients[(l, m)] * special.sph_harm(m, l, phi, theta)
    return approx

# Generate a meshgrid for theta and phi
theta_mesh, phi_mesh = np.meshgrid(np.linspace(0, np.pi, 100), np.linspace(0, 2*np.pi, 100))

# Evaluate the original function and the approximation on the meshgrid
original_values = function_spherical(theta_mesh, phi_mesh)
approximation_values = spherical_harmonics_approx(theta_mesh, phi_mesh, coefficients_spherical, l_max)

# Convert to Cartesian coordinates for plotting
x_original = np.sin(theta_mesh) * np.cos(phi_mesh) * original_values.real
y_original = np.sin(theta_mesh) * np.sin(phi_mesh) * original_values.real
z_original = np.cos(theta_mesh) * original_values.real

x_approx = np.sin(theta_mesh) * np.cos(phi_mesh) * approximation_values.real
y_approx = np.sin(theta_mesh) * np.sin(phi_mesh) * approximation_values.real
z_approx = np.cos(theta_mesh) * approximation_values.real

# Create plots
fig = plt.figure(figsize=(12, 6))

# Original function
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(x_original, y_original, z_original, cmap='viridis')
ax1.set_title('Original Function')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

# Spherical harmonics approximation
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(x_approx, y_approx, z_approx, cmap='viridis')
ax2.set_title('Spherical Harmonics Approximation')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')

plt.show()



# Plot color difference

In [13]:
%matplotlib qt
#%matplotlib inline
# Function to plot a function on the surface of a sphere using color
def plot_function_on_sphere(ax, theta, phi, values, title):
    # Convert to Cartesian coordinates for plotting
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    # 
    # x = abs(values) * np.sin(theta) * np.cos(phi)
    # y = abs(values) * np.sin(theta) * np.sin(phi)
    # z = abs(values) * np.cos(theta)

    # x = values * np.sin(theta) * np.cos(phi)
    # y = values * np.sin(theta) * np.sin(phi)
    # z = values * np.cos(theta)

    # Plot the function values as colors on the sphere
    ax.plot_surface(x, y, z, facecolors=plt.cm.viridis(values), rstride=1, cstride=1)
    ax.set_title(title)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

# Create plots
fig = plt.figure(figsize=(12, 6))

# Original function
ax1 = fig.add_subplot(121, projection='3d')
plot_function_on_sphere(ax1, theta_mesh, phi_mesh, original_values.real, 'Original Function')

# Spherical harmonics approximation
ax2 = fig.add_subplot(122, projection='3d')
plot_function_on_sphere(ax2, theta_mesh, phi_mesh, approximation_values.real, 'Spherical Harmonics Approximation')

plt.show()



# Ugly function expansion using SHtools

In [ ]:
import numpy as np
import pyshtools

# Step 1: Define the function on a grid over the sphere
def func(theta, phi):
    return (np.sin(theta)**3 * np.cos(phi)**3) / np.exp(np.sin(theta) * np.sin(phi))

# Define the grid parameters
lmax = 15  # Maximum degree of spherical harmonics
theta = np.linspace(0, np.pi, 2 * lmax + 1)  # Colatitude
phi = np.linspace(0, 2 * np.pi, 2 * lmax + 1)  # Longitude
theta, phi = np.meshgrid(theta, phi)

# Evaluate the function on the grid
f_grid = func(theta, phi)

# Step 2: Expand the grid into spherical harmonics coefficients
coeffs = pyshtools.expand.SHExpandDH(f_grid, sampling=2)

# Step 3 (optional): Reconstruct the function from these coefficients
f_reconstructed = pyshtools.expand.MakeGridDH(coeffs, sampling=2)

coeffs.shape, f_reconstructed.shape  # Display the shape of the coefficients and the reconstructed function

